[![Image Name](https://cdn.kesci.com/upload/sehv2zq3qx.jpg?imageView2/0/w/960/h/960)](https://www.heywhale.com/home/competition/66598b3271a1fd975a17d6ad)  
[**vgbhfive**](http://blog.vgbhfive.com)，多年风控引擎研发及金融模型开发经验，现任某公司风控研发工程师，对数据分析、金融模型开发、风控引擎研发具有丰富经验。

在前一关我们学习了逻辑回归，学会如何训练模型、数据基础性分析、如何处理空值等操作，下面我们开始新的一关 `KMeans`。

### KMeans  
`KMeans` 是我们最常用的基于欧式距离的聚类算法，其认为两个目标的距离越近，相似度越大。  

`KMeans` 算法的思想很简单，对于给定的样本集，按照样本之间的距离大小，将样本集划分为 `K` 个簇，其目的是让簇内的点尽量紧密的连在一起，而让簇间的距离尽量的大。

### 基于 `KMeans` 的股票分类

以往的量化投资中对于股票的划分分类，通常取决于行业、市值、地域等等指标划分，而这些分类指标并不能很好的区分公司的好坏。而现在可以通过每日的交易行情实时划分分类，通过计算当日前一个月的分类从而确定该股票分类，更好的降低投资风险，提供风险对冲。该数据集有 `2024-05-06` 的全部上市公司股票交易行情信息，其中包含日期、开盘价、收盘价、最高价、最低价、成交量、成交额等特征信息，另外该模型使用的数据为真实数据，可以在实际操作中使用。  

股市有风险，入市需谨慎！

#### 引入依赖

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, silhouette_score

#### 加载数据

In [3]:
# 1. 加载数据

stock = pd.read_csv('/home/mw/input/stock_202405066648/2024-05-06.csv', index_col='Unnamed: 0')
stock.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/mw/input/stock_202405066648/2024-05-06.csv'

In [ ]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5360 entries, 0 to 5359
Data columns (total 20 columns):
symbol           5360 non-null object
code             5360 non-null int64
name             5360 non-null object
trade            5360 non-null float64
pricechange      5360 non-null float64
changepercent    5360 non-null float64
buy              5360 non-null float64
sell             5360 non-null float64
settlement       5360 non-null float64
open             5360 non-null float64
high             5360 non-null float64
low              5360 non-null float64
volume           5360 non-null int64
amount           5360 non-null int64
ticktime         5360 non-null object
per              5360 non-null float64
pb               5360 non-null float64
mktcap           5360 non-null float64
nmc              5360 non-null float64
turnoverratio    5360 non-null float64
dtypes: float64(14), int64(3), object(3)
memory usage: 879.4+ KB


In [ ]:
# 2. 删除与分类数无关的特征列

new_stock = stock.drop(['symbol', 'code', 'name', 'ticktime'], axis=1)
new_stock.head()

,trade,pricechange,changepercent,buy,sell,settlement,open,high,low,volume,amount,per,pb,mktcap,nmc,turnoverratio
0,10.89,0.10,0.927,10.88,10.89,10.79,10.96,11.11,10.84,178410057,1953817493,4.840,0.508,2.113304e+07,2.113264e+07,0.91938
1,7.46,0.05,0.675,7.45,7.46,7.41,7.63,7.88,7.44,524493788,3996921703,7.243,0.355,8.900309e+06,7.248834e+06,5.39773
2,10.44,0.23,2.253,10.43,10.44,10.21,9.99,10.46,9.97,9885440,102059842,-8.821,7.478,1.382050e+05,1.318448e+05,7.82769
3,3.87,0.00,0.000,3.87,3.88,3.87,3.96,4.01,3.86,23187186,91348765,-6.509,0.752,5.224481e+05,5.224451e+05,1.71759
4,4.09,-0.15,-3.538,4.09,4.10,4.24,4.26,4.26,4.04,2440550,10028589,39.403,11.212,1.416972e+05,1.263597e+05,0.78995


#### 确定分类个数

In [ ]:
# 3. 利用肘部法则确定分类数

inertia = []
silhouette_scores = []
i_range = range(2, 11)
for i in i_range:
    kmeans = KMeans(n_clusters=i, random_state=10).fit(new_stock)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(new_stock, kmeans.labels_))

inertia, silhouette_scores

([4.145014955246118e+20,
  2.189240016626829e+20,
  1.3680212142259103e+20,
  9.515375759658358e+19,
  6.825673805881965e+19,
  5.212205817545241e+19,
  3.9704368251938726e+19,
  3.1151400556240667e+19,
  2.459673146703804e+19],
 [0.8944521948807374,
  0.8264604737729477,
  0.7483080588736405,
  0.7481055442872104,
  0.6631973213181866,
  0.6633331494897194,
  0.6324316985420916,
  0.6193693983202632,
  0.6209177252260801])

In [ ]:
# 4. 确定分类数
plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(i_range, inertia, marker='o')

plt.subplot(1, 2, 2)
plt.plot(i_range, silhouette_scores, marker='o')

plt.tight_layout()
plt.show()

# 左图在 2 到 5 的时候，曲线下降速率明显下降。
# 右图在 2，3，4，5 时，轮廓系数比较高。
# 结合两图，选择 3 作为聚类数。

<Figure size 1080x360 with 2 Axes>

In [ ]:
# 5. 分类

kmeans_final = KMeans(n_clusters=3, random_state=10).fit(new_stock)

labels = kmeans_final.labels_
new_stock['cluster'] = labels

#### 查看分类结果

In [ ]:
# 6. 查看分类情况

new_stock['cluster'].value_counts()

0    5001
1     329
2      30
Name: cluster, dtype: int64

In [ ]:
print(new_stock.shape)
print(stock.shape)

(5360, 17)
(5360, 20)


## 按题目要求 分类数设为 4

In [ ]:
# # kmeans_final = KMeans(n_clusters=4, random_state=10).fit(new_stock)

# # labels = kmeans_final.labels_
# # new_stock['cluster'] = labels
# # target_index = stock[stock['symbol'] == 'sz002829'].index
# # print(new_stock.loc[target_index, 'cluster'])
# # print(stock.loc[target_index, 'symbol'])
# new_stock = stock.drop(['symbol', 'name', 'ticktime'], axis=1)
# kmeans_final = KMeans(n_clusters=4, random_state=10).fit(new_stock)
# labels = kmeans_final.labels_
# new_stock['cluster'] = labels
# print(stock[new_stock['code'] == 2829]['cluster'])
data_tmp = stock.copy()

new_data_tmp = data_tmp.drop(['symbol', 'name', 'ticktime'], axis=1)

kmeans_final = KMeans(n_clusters=4, random_state=42).fit(new_data_tmp) # 试试  random_state=42

labels = kmeans_final.labels_
new_data_tmp['cluster'] = labels

new_data_tmp[new_data_tmp['code'] == 2829]['cluster']

NameError: name 'stock' is not defined

In [ ]:
# 查看分类情况

new_data_tmp['cluster'].value_counts()

0    4570
2     632
1     131
3      27
Name: cluster, dtype: int64

#### 总结  
`KMeans` 在确定分类个数计算时，无法使用 `object` 类型的数据，应当提前删除或对特征进行 `one-hot` 处理。  



In [ ]:
# 利用肘部法则确定前300个记录的分类数

inertia = []
silhouette_scores = []
i_range = range(2, 11)
subset_new_stock = new_stock.iloc[0:300]
for i in i_range:
    kmeans = KMeans(n_clusters=i, random_state=10).fit(subset_new_stock)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(subset_new_stock, kmeans.labels_))

inertia, silhouette_scores

([2.5285859868686787e+19,
  1.2952821174611915e+19,
  7.410791790089599e+18,
  4.766749141417961e+18,
  3.193017605109102e+18,
  2.3343124574636585e+18,
  1.9598314013000297e+18,
  1.6228287900652856e+18,
  1.3752345331260751e+18],
 [0.8972361973100779,
  0.7968344475920585,
  0.7015820307845293,
  0.6555643062015785,
  0.6549870304348072,
  0.6066647440992684,
  0.6040674330096649,
  0.5857200751844136,
  0.5685142900972827])

In [ ]:
# 4. 确定子集的分类数
plt.figure(figsize=(15,5))

plt.subplot(1, 2, 1)
plt.plot(i_range, inertia, marker='o')

plt.subplot(1, 2, 2)
plt.plot(i_range, silhouette_scores, marker='o')

plt.tight_layout()
plt.show()

# 左图在 2 到 5 的时候，曲线下降速率明显下降。
# 右图在 2，3，4，5 时，轮廓系数比较高。
# 结合两图，选择 3 作为聚类数。

<Figure size 1080x360 with 2 Axes>

我觉的3比较合适

### 闯关题

#### STEP1：请根据要求完成题目  


Q1. KMeans 中某个参数的含义是正确的？  
   A. n_clusters 分类个数  
   B. inertia_ 轮廓系数  
   C. silhouette_scores 曲线下降速率  

Q2. 修改KMeans的划分集群个数为 4个，那么 002829 股票的分类是哪个？  
   A. 0  
   B. 1  
   C. 2  
   D. 3  

Q3. 前300个股票数据集划分集群的最优个数是多少？  
   A. 1  
   B. 3  
   C. 5  
   D. 10

In [ ]:
new_stock = new_stock[0:300]

inertia = []
silhouette_scores = []
i_range = range(2, 11)
for i in i_range:
    # 计算分类并保存指标

inertia, silhouette_scores

In [ ]:
#填入你的答案并运行,注意大小写
a1 = 'A'  # 如 a1= 'A'
a2 = 'A'  # 如 a2= 'A'
a3 = 'B'  # 如 a3= 'A'

#### STEP2：将结果保存为 csv 文件  
将结果保存为 csv 文件  
csv 需要有两列，列名：id、answer。其中，id 列为题号，如 q1、q2；answer 列为 STEP1 中各题你计算出来的结果。💡 这一步的代码你不用做任何修改，直接运行即可。

In [ ]:
import pandas as pd

# 生成 csv 作业答案文件
def save_csv(a1, a2, a3):
    df = pd.DataFrame({"id": ["q1", "q2", "q3"], "answer": [a1, a2, a3]})
    df.to_csv("answer_2.csv", index=None)
    print(df)

save_csv(a1,a2,a3)

   id answer
0  q1      A
1  q2      A
2  q3      B


#### STEP3: 提交 csv 文件，获取分数结果  
提交 csv 文件，获取分数结果  

你的 csv 答案文件已经准备完毕了，最后让我们提交答案文件，看看是否正确。  

提交方法：  

1、拷贝提交 token  

去对应关卡的 提交页面，找到对应关卡，看到了你的 token 嘛？  

拷贝它。  

记得：每个关卡的 token 不一样。  

2、下方 cell 里，拿你拷贝的 token 替换掉 XXXXXXX， 然后 Ctrl+Enter 运行 。  



In [ ]:
#运行这个Cell 下载提交工具

!wget -nv -O heywhale_submit https://cdn.kesci.com/submit_tool/v4/heywhale_submit&&chmod +x heywhale_submit

# 运行提交工具
# 把下方XXXXXXX替换为你的 Token
# 改完看起来像是：!./heywhale_submit -token 586eeef71cb92941 -file answer_2.csv

!./heywhale_submit -token 47c33e411537dd5c -file answer_2.csv  # 替换XXXXXXX；注意不可增减任何空格或其他字符

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
2025-07-18 03:34:27 URL:https://cdn.kesci.com/submit_tool/v4/heywhale_submit [22020102/22020102] -> "heywhale_submit" [1]
Heywhale Submit Tool: v5.0.1 
> 已验证Token
> 开始上传文件
25 / 25 [||||||||||||||||||||||||||||||||||||||||||||||||||||||||] ? p/s 100.00%
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
